In [1]:
# 추천 기준: 공원, 영화관
# 집값: 1억~3억
# 근무지: 숙명여대
# 통근시간: 버스or 지하철 60분 이내 

In [16]:
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_similarity
import time
import json
import os
import ssl
import urllib.request
#df = pd.read_csv('./train_all.csv')

In [83]:
def transit_api(address, transit, origin_list): #
    destination = address
    mode = 'transit'
    transit_mode =transit[0]
    limit_time = transit[1]
    
    filtered_idx = {}
    idx_content={}


    #if (__name__ == "__main__"):
        #client = None
    with open("./google_key.json","r") as clientJson :
        client = json.load(clientJson)
        print('완료1')
        print(origin_list)

    for origin in origin_list:
        if len(filtered_idx) < 3:

            departure_time  = "1640818800" #2021 12 30 AM8:00 TIMESTAMP
            key             = client['key']
            print('완료2')

            url = "https://maps.googleapis.com/maps/api/directions/json?origin="+ origin + "&destination=" + destination + "&mode=" + mode+ "&transit_mode="+ transit_mode + "&departure_time=" + departure_time + "&language=ko"+ "&key=" + key

            request         = urllib.request.Request(url)
            context         = ssl._create_unverified_context()
            response        = urllib.request.urlopen(request, context=context)
            responseText    = response.read().decode('utf-8')
            content    = json.loads(responseText)
            #print(content)
            path   = content["routes"][0]["legs"][0]
            duration_sec  = path["duration"]["value"]
            #print(duration_sec)
            if duration_sec <= (limit_time*60):
                filtered_idx[origin] = duration_sec
                print('완료3')
                idx_content[len(filtered_idx)] = content
                with open("./result2.json","w") as json_file:
                    json.dump(idx_content, json_file, indent=4)
                    print('완료4')
    return filtered_idx 

def recommend(address, price, transit, options): #['영화관','도서관']
    df = pd.read_csv('./train_all.csv')
    col = ['address']
    for i in range(len(options)):
        col.append(options[i])
    print(col)
    for i in range(len(col)):
        idx = df[df[col[i]] == 0].index
        df=df.drop(idx)
    data = df.copy()

    data = df.copy()
    data = data[col]
    data = data.set_index('address')
    
    data_max= data.iloc[:,:5].max(axis=0)
    data_max['address'] = '비교'
    data = data.reset_index(drop=False)
    data = data[col].append(data_max, ignore_index=True)
    data = data.set_index('address')
    
    col.append('공시가격')
    col.append('좌표')
    data_ = df[col]

    data_max_= data_.iloc[:,:6].max(axis=0)
    data_max_['address'] = '비교'
    data_ = data_[col].append(data_max_, ignore_index=True)
    data_ = data_.set_index('address')
    
    
    scaler = preprocessing.MinMaxScaler()
    data[data.columns] = scaler.fit_transform(data[data.columns])
    print(data)
    euclidean = euclidean_distances(data ,data)
    distance = euclidean[-1, :].reshape(-1)
    
    data = data.reset_index(drop=False)
    index_distance = {}
    for idx, val in enumerate(distance):
        index_distance[idx] = val

    for i in range(len(index_distance)):
        #index = index_distance[i][0]
        distance = index_distance[i]
        data.loc[i,'distance'] = distance
    dc = data.sort_values(by='distance', ascending=True)
    dc = dc.set_index('address')
    print(dc)
    data2 = dc[['distance']]
    
    data2 = data2.join(data_)
    data2 = data2.fillna(0)
    comp = data2.iloc[0]
    print(comp)
    
    data2 = data2.drop('비교')
    print('data2', data2)
    
    #price = [1,3]
    condition = (data2['공시가격'] > price[0]) &(data2['공시가격'] < price[1])
    price_data = data2[condition]
    
    origin_list = price_data['좌표'].tolist()
    print(origin_list)
    filtered_idx = transit_api(address, transit, origin_list)
    print(filtered_idx)
    
    comp = comp.to_frame()
    comp =comp.T
    for i in range(len(filtered_idx)):
        comp = comp.append(price_data[(price_data['좌표']==(list(filtered_idx.keys())[i]))])
    
    comp['공시가격'][0] = price
    comp['좌표'][0] = address
    comp['교통편']=0
    trans = list(filtered_idx.values())
    for i in range(len(filtered_idx)):
        comp['교통편'][i+1]= trans[i]
    comp['교통편'][0] = transit[1]*60
    #comp.to_csv('./교통추천결과값4.csv',encoding='euc-kr')
    result = comp.to_json()
    print(comp)
    return result
    

    #destination = '37.546475,126.964692'
    #mode = 'transit'
    #transit_mode = 'bus|subway'
    #limit_time = 60
    


In [84]:
#def recommend(address, price, transit, options)
result = recommend('37.546475,126.964692', [0,3000000000], ['bus', 60], ['공원','영화관','초등학교'])

['address', '공원', '영화관', '초등학교']
                     공원       영화관      초등학교
address                                    
서울특별시 강남구 논현동  0.161290  1.000000  0.071429
서울특별시 강남구 도곡동  0.258065  0.000000  0.071429
서울특별시 강남구 삼성동  0.064516  0.166667  0.071429
서울특별시 강남구 신사동  0.000000  0.333333  0.000000
서울특별시 강남구 역삼동  0.322581  0.583333  0.071429
...                 ...       ...       ...
인천광역시 계양구 작전동  0.161290  0.166667  0.214286
인천광역시 서구 연희동   0.064516  0.000000  0.000000
인천광역시 서구 당하동   0.000000  0.000000  0.142857
인천광역시 서구 청라동   0.354839  0.083333  0.428571
비교             1.000000  1.000000  1.000000

[135 rows x 3 columns]
                       공원       영화관      초등학교  distance
address                                                
비교               1.000000  1.000000  1.000000  0.000000
인천광역시 연수구 송도동    1.000000  0.166667  1.000000  0.833333
경기도 시흥시 정왕동      0.838710  0.500000  0.142857  1.005340
인천광역시 남동구 논현동    0.354839  0.166667  0.571429  1.137695
경기도 부천시 중동       0.451613  0.166667

완료2
완료2
완료2
완료2
완료2
완료2
완료2
완료2
완료3
완료4
완료2
완료2
완료3
완료4
완료2
완료2
완료2
완료2
완료2
완료2
완료2
완료2
완료2
완료2
완료2
완료2
완료2
완료2
완료3
완료4
{'37.500335,127.037596': 3589, '37.621298,127.024902': 3344, '37.476412,126.925691': 3439}
               distance  공원 영화관 초등학교             공시가격                    좌표  \
비교                  0.0  32  13   15  [0, 3000000000]  37.546475,126.964692   
서울특별시 강남구 역삼동  1.222601  11   8    2      819200531.3  37.500335,127.037596   
서울특별시 강북구 미아동  1.240724   7   4    7      461559225.9  37.621298,127.024902   
서울특별시 관악구 신림동  1.336064   6   1   11      443031953.8  37.476412,126.925691   

                교통편  
비교             3600  
서울특별시 강남구 역삼동  3589  
서울특별시 강북구 미아동  3344  
서울특별시 관악구 신림동  3439  


<ipython-input-83-154edd1a8e19>:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp['교통편'][i+1]= trans[i]
<ipython-input-83-154edd1a8e19>:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp['교통편'][0] = transit[1]*60


In [35]:
comp

,distance,공원,영화관,공시가격,좌표,교통편
비교,0.0,32,13,"[1, 3]","37.546475,126.964692",3600
경기도 고양시 일산동구 백석동,1.228927,4,3,191598465.0,"37.644500,126.786140",3351
서울특별시 중구 을지로6가,1.301708,1,3,193672391.3,"37.566868,127.006310",1540
서울특별시 강북구 수유동,1.301708,1,3,293740725.3,"37.639388,127.017183",3076


In [36]:
comp.to_csv('./교통추천결과값4.csv',encoding='euc-kr')